In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

class FeatureClassifier:
  def __init__(self,reqAcc=0.01,classifier='DesicionTree',bias=[],control=None,n_jobs=None,random_state=None):
    self.featureClassifiers=[]
    self.reqAcc=reqAcc
    self.indexLs=[]
    self.flag=0
    self.bias=bias
    self.control=control
    self.classifier=classifier
    self.dic={'DesicionTree':0,'LinearRegression':1,'SVM':2,'LogisticRegression':3}
    self.n_jobs=n_jobs
    self.random_state=random_state
    self.num_lables=2
  def finIndex(self):
    for i in range(len(self.featureClassifiers)):
      if self.featureClassifiers[i][1] < self.reqAcc:
        return i
      self.indexLs.append(self.featureClassifiers[i][2])
    self.flag=1
    return i
  def fit(self,x,y):
    self.num_lables=len(set(y.flatten()))
    bestfeatures = SelectKBest(score_func=chi2,k=1)
    fit = bestfeatures.fit(x,y)
    summ=sum(fit.scores_)
    for i in range(len(x[0])):
      clf=[DecisionTreeClassifier(max_depth=self.control,random_state=self.random_state),LinearRegression(n_jobs=self.n_jobs),SVC(gamma=self.control,random_state=self.random_state), LogisticRegression(penalty=self.control,random_state=self.random_state)][self.dic[self.classifier]]
      X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
      clf.fit(X_train[:,i:i+1],y_train)
      self.featureClassifiers.append((clf,fit.scores_[i]/summ,i))
    self.featureClassifiers.sort(key=lambda x:x[1],reverse=True)
    index=self.finIndex()
    if self.flag==0:
      self.featureClassifiers=self.featureClassifiers[:index]
    else:
      pass
    return
  def predict(self,x):
    if not self.bias:
      self.bias=np.zeros(self.num_lables)
    if len(self.bias)<self.num_lables:
      raise AttributeError('Please check the lenth of bias list')
    yPred=[]
    for i in range(len(x)):
      pred_arr=np.zeros(self.num_lables)
      for j in range(len(self.indexLs)):
        pred=np.round(self.featureClassifiers[j][0].predict([[x[i][self.indexLs[j]]]]))
        pred_arr[pred]+=self.featureClassifiers[j][1]+self.bias[pred[0]]
      yPred.append(np.argmax(pred_arr))
    return yPred

In [ ]:
df=pd.read_csv("/content/c1908combi.csv")
#df.drop(labels=["nets"],axis=1,inplace=True)

,nets,level,connectivity,pi,po,score,Fanout,Fanin,Load,Resistance,Pins,Net load,Prob.,Rate,Power,label
0,I2,1,1,1,39,42,1,1,0.24,0.02,2,0.243,0.242,0.0980,0.0058,1
1,I3,2,1,2,38,43,1,1,0.24,0.02,2,0.243,0.242,0.0980,0.0058,1
2,N1,0,2,0,40,42,4,1,1.20,0.09,5,0.243,0.244,0.1372,0.0082,1
3,N10,0,3,0,40,43,5,1,1.56,0.12,6,0.243,0.244,0.1372,0.0082,1
4,N1001,3,1,3,37,44,1,1,0.24,0.02,2,0.243,0.750,0.0980,0.0058,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,N992,3,1,3,37,44,1,1,0.24,0.02,2,0.243,0.870,0.0710,0.0042,0
911,N993,3,1,3,37,44,1,1,0.24,0.02,2,0.243,0.870,0.0710,0.0042,0
912,N994,3,2,3,37,45,1,1,0.24,0.02,2,0.243,0.870,0.0710,0.0042,0
913,N997,3,1,3,37,44,1,1,0.24,0.02,2,0.243,0.870,0.0710,0.0042,0


In [ ]:
df.dropna(inplace = True)
finLis=[]
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
try:
  X = df.drop(['nets','Label'],axis=1).values
  y = df['Label'].values
except:
  X = df.drop(['nets','label'],axis=1).values
  y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
clf1=FeatureClassifier(0,classifier='DesicionTree',control=3)
clf1.fit(X_train,y_train.reshape(-1,1))

In [ ]:
print(accuracy_score(y_test,clf1.predict(X_test)))

0.9768211920529801


In [ ]:
print(clf1.indexLs)
clf1.featureClassifiers

[5, 9, 7, 10, 1, 0, 2, 3, 8, 13, 11, 4, 12, 6]


[(DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                         max_depth=3, max_features=None, max_leaf_nodes=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
                         min_weight_fraction_leaf=0.0, presort='deprecated',
                         random_state=None, splitter='best'),
  0.32434136377857187,
  5),
 (DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                         max_depth=3, max_features=None, max_leaf_nodes=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
                         min_weight_fraction_leaf=0.0, presort='deprecated',
                         random_state=None, splitter='best'),
  0.20759370905381277,
  9),
 (DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion

In [ ]:
# clf=DecisionTreeClassifier()
# clf=clf.fit(X_test[:,0:1],y_test)
# print(accuracy_score(y_test,clf.predict(X_test[:,0:1])))